delete existing index

In [1]:
curl -XDELETE 'localhost:9200/my_index?pretty'

{
  "acknowledged" : true
}


for long complicated words, we could use a compound word token filter. However, your results will only be as good as the dictionary

Gonna try a n-gram again, with a trigram

In [2]:
curl -XPUT 'localhost:9200/my_index?pretty' -H 'Content-Type: application/json' -d'
{
    "settings": {
        "analysis": {
            "filter": {
                "trigrams_filter": {
                    "type":     "ngram",
                    "min_gram": 3,
                    "max_gram": 3
                }
            },
            "analyzer": {
                "trigrams": {
                    "type":      "custom",
                    "tokenizer": "standard",
                    "filter":   [
                        "lowercase",
                        "trigrams_filter"
                    ]
                }
            }
        }
    },
    "mappings": {
        "my_type": {
            "properties": {
                "text": {
                    "type":     "string",
                    "analyzer": "trigrams" 
                }
            }
        }
    }
}
'

{
  "acknowledged" : true,
  "shards_acknowledged" : true
}


test out the new analyzer

In [3]:
curl -XGET 'localhost:9200/my_index/_analyze?analyzer=trigrams&pretty' -H 'Content-Type: application/json' -d'
Weißkopfseeadler
'

{
  "tokens" : [
    {
      "token" : "wei",
      "start_offset" : 1,
      "end_offset" : 17,
      "type" : "word",
      "position" : 0
    },
    {
      "token" : "eiß",
      "start_offset" : 1,
      "end_offset" : 17,
      "type" : "word",
      "position" : 0
    },
    {
      "token" : "ißk",
      "start_offset" : 1,
      "end_offset" : 17,
      "type" : "word",
      "position" : 0
    },
    {
      "token" : "ßko",
      "start_offset" : 1,
      "end_offset" : 17,
      "type" : "word",
      "position" : 0
    },
    {
      "token" : "kop",
      "start_offset" : 1,
      "end_offset" : 17,
      "type" : "word",
      "position" : 0
    },
    {
      "token" : "opf",
      "start_offset" : 1,
      "end_offset" : 17,
      "type" : "word",
      "position" : 0
    },
    {
      "token" : "pfs",
      "start_offset" : 1,
      "end_offset" : 17,
      "type" : "word",
      "position" : 0
    },
    {
      "token" : "fse",
      "start_offset" : 1,
      "end_

throw in some test data

In [4]:
curl -XPOST 'localhost:9200/my_index/my_type/_bulk?pretty' -H 'Content-Type: application/json' -d'
{ "index": { "_id": 1 }}
{ "text": "Aussprachewörterbuch" }
{ "index": { "_id": 2 }}
{ "text": "Militärgeschichte" }
{ "index": { "_id": 3 }}
{ "text": "Weißkopfseeadler" }
{ "index": { "_id": 4 }}
{ "text": "Weltgesundheitsorganisation" }
{ "index": { "_id": 5 }}
{ "text": "Rindfleischetikettierungsüberwachungsaufgabenübertragungsgesetz" }
'

{
  "took" : 94,
  "errors" : false,
  "items" : [
    {
      "index" : {
        "_index" : "my_index",
        "_type" : "my_type",
        "_id" : "1",
        "_version" : 1,
        "result" : "created",
        "_shards" : {
          "total" : 2,
          "successful" : 2,
          "failed" : 0
        },
        "created" : true,
        "status" : 201
      }
    },
    {
      "index" : {
        "_index" : "my_index",
        "_type" : "my_type",
        "_id" : "2",
        "_version" : 1,
        "result" : "created",
        "_shards" : {
          "total" : 2,
          "successful" : 2,
          "failed" : 0
        },
        "created" : true,
        "status" : 201
      }
    },
    {
      "index" : {
        "_index" : "my_index",
        "_type" : "my_type",
        "_id" : "3",
        "_version" : 1,
        "result" : "created",
        "_shards" : {
          "total" : 2,
          "successful" : 2,
          "failed" : 0
        },
        "created" : tru

simple test for Adler

In [5]:
curl -XGET 'localhost:9200/my_index/my_type/_search?pretty' -H 'Content-Type: application/json' -d'
{
    "query": {
        "match": {
            "text": "Adler"
        }
    }
}
'

{
  "took" : 5,
  "timed_out" : false,
  "_shards" : {
    "total" : 5,
    "successful" : 5,
    "failed" : 0
  },
  "hits" : {
    "total" : 1,
    "max_score" : 0.56437,
    "hits" : [
      {
        "_index" : "my_index",
        "_type" : "my_type",
        "_id" : "3",
        "_score" : 0.56437,
        "_source" : {
          "text" : "Weißkopfseeadler"
        }
      }
    ]
  }
}


minimum should match should reduce the number of trivial results, by requiring a minimum number of trigrams to be present

In [6]:
curl -XGET 'localhost:9200/my_index/my_type/_search?pretty' -H 'Content-Type: application/json' -d'
{
    "query": {
        "match": {
            "text": {
                "query":                "Gesundheit",
                "minimum_should_match": "80%"
            }
        }
    }
}
'

{
  "took" : 3,
  "timed_out" : false,
  "_shards" : {
    "total" : 5,
    "successful" : 5,
    "failed" : 0
  },
  "hits" : {
    "total" : 3,
    "max_score" : 0.48138323,
    "hits" : [
      {
        "_index" : "my_index",
        "_type" : "my_type",
        "_id" : "5",
        "_score" : 0.48138323,
        "_source" : {
          "text" : "Rindfleischetikettierungsüberwachungsaufgabenübertragungsgesetz"
        }
      },
      {
        "_index" : "my_index",
        "_type" : "my_type",
        "_id" : "4",
        "_score" : 0.38452482,
        "_source" : {
          "text" : "Weltgesundheitsorganisation"
        }
      },
      {
        "_index" : "my_index",
        "_type" : "my_type",
        "_id" : "2",
        "_score" : 0.29822114,
        "_source" : {
          "text" : "Militärgeschichte"
        }
      }
    ]
  }
}
